<a href="https://colab.research.google.com/github/abakashi/dinamicas_dex_days/blob/main/Wrangling_Case1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
#Import das bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [100]:
#Carregamento dos raw datasets nos dataframes
rwdf_churn = pd.read_csv('/content/drive/MyDrive/Datasets/wranglingcase1/churn.csv', dtype={'employee': int, 'numVotes': int, 'stillExists': bool})
rwdf_votes = pd.read_csv('/content/drive/MyDrive/Datasets/wranglingcase1/votes.csv')
rwdf_cmm_int = pd.read_csv('/content/drive/MyDrive/Datasets/wranglingcase1/commentInteractions.csv', dtype={'employee': int, 'liked': int, 'disliked': int})
rwdf_cmm_clean = pd.read_csv('/content/drive/MyDrive/Datasets/wranglingcase1/comments_clean_anonimized.csv')

In [101]:
#Começando o estudo do Churn

# Criando uma função para colocar as datas num formato mais palatável.
def time_formatter(series):
  dates = list(series)
  tmz = list()
  dts = list()

  for date in dates:
    if date[-7] == 'S':
      day = date[8:10]
      mon = date[4:7]
      yr = date[-4:]
      time = date[11:19]
      dts.append(f'{day}/{mon}/{yr} - {time}')
      tmz.append('CEST')
    else:
      day = date[8:10]
      mon = date[4:7]
      yr = date[-4:]
      time = date[11:19]
      dts.append(f'{day}/{mon}/{yr} - {time}')
      tmz.append('CET')
  return (pd.Series(dts),pd.Series(tmz))

#criando um novo dataset vazio
rwdf_churn_date_form = pd.DataFrame()

#implementando as duas colunas referentes à data
rwdf_churn_date_form['LPDate_formated'], rwdf_churn_date_form['LPDate_tmz'] = time_formatter(rwdf_churn['lastParticipationDate'])

#convertendo a coluna para o formato datetime.
rwdf_churn_date_form['LPDate_formated'] = pd.to_datetime(rwdf_churn_date_form['LPDate_formated'], format='%d/%b/%Y -  %H:%M:%S')

#criando as demais colunas no novo dataset
rwdf_churn_date_form['employee'], rwdf_churn_date_form['com_alias'], rwdf_churn_date_form['votes'], rwdf_churn_date_form['still_exists']  = rwdf_churn['employee'], rwdf_churn['companyAlias'], rwdf_churn['numVotes'], rwdf_churn['stillExists']

#adicionando + uma hora de delta nos timezones CEST (horario de verão europeu)
rwdf_churn_date_form.loc[rwdf_churn_date_form['LPDate_tmz'] == 'CEST', 'LPDate_formated'] = rwdf_churn_date_form['LPDate_formated'] + pd.Timedelta('1 hour')

#associando o conjunto novo com o antigo
rwdf_churn = rwdf_churn_date_form

In [102]:
#checando por valores nulos
rwdf_churn.isna().sum()
#nenhum valor nulo

#checando valores duplicados
rwdf_churn.duplicated().sum()
#sem linhas duplicadas

rwdf_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   LPDate_formated  4847 non-null   datetime64[ns]
 1   LPDate_tmz       4847 non-null   object        
 2   employee         4847 non-null   int64         
 3   com_alias        4847 non-null   object        
 4   votes            4847 non-null   int64         
 5   still_exists     4847 non-null   bool          
dtypes: bool(1), datetime64[ns](1), int64(2), object(2)
memory usage: 194.2+ KB


In [103]:
#Analisando o conjunto votes

#Aplicando a time_formatter nas datas
rwdf_votes['vDate_form'], rwdf_votes['vDate_tmz'] = time_formatter(rwdf_votes['voteDate'])

#convertendo para formato datetime
rwdf_votes['vDate_form'] = pd.to_datetime(rwdf_votes['vDate_form'], format='%d/%b/%Y -  %H:%M:%S')

#adicionando mais uma hora nos horários em horário de verão
rwdf_votes.loc[rwdf_votes['vDate_tmz'] == 'CEST', 'vDate_form'] = rwdf_votes['vDate_form'] + pd.Timedelta('1 hour')

#apagando a coluna de data antiga
rwdf_votes.drop('voteDate', axis=1, inplace=True)

In [104]:
#Verificando valores nulos: Sem valores nulos
rwdf_votes.isna().sum()

#Verificando linhas duplicadas: 87 Linhas duplicadas encontradas.
rwdf_votes.duplicated().sum()

rwdf_votes = rwdf_votes[~rwdf_votes.duplicated()]

#Linhas duplicadas eliminadas.
rwdf_votes.duplicated().sum()

0

In [105]:
#verificando o conjunto de c interações de comentários

#verificando valores nulos: valores nulos encontrados na ultima linha do conjunto
rwdf_cmm_int.isna().sum()

#Eliminando a ultima linha por não ser significante.
rwdf_cmm_int.drop(336959, axis=0, inplace=True)

In [106]:
#Convertendo colunas liked e disliked para booleano.
rwdf_cmm_int = rwdf_cmm_int.astype({'liked': int, 'disliked': int})

#verificando de as colunas liked e disliked são realmente redundantes.

lik_dis = zip(tuple(rwdf_cmm_int['liked']), tuple(rwdf_cmm_int['disliked']))

for ocrr in tuple(lik_dis):
  if ocrr == (0, 0) or ocrr == (1, 1):
    print(False)
#As colunas são mesmo redundantes

In [107]:
#Removendo a coluna disliked
rwdf_cmm_int.drop('disliked', axis=1, inplace=True)

In [108]:
rwdf_cmm_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336959 entries, 0 to 336958
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   employee      336959 non-null  int64 
 1   companyAlias  336959 non-null  object
 2   liked         336959 non-null  int64 
 3   commentId     336959 non-null  object
dtypes: int64(2), object(2)
memory usage: 12.9+ MB


In [109]:
#verificando comments clean annotations

#removendo ultima linha que continha maior parte dos atributos nulos
rwdf_cmm_clean.drop(82755, axis=0, inplace=True)

In [110]:
#Convertendo Likes e Dislikes para int
rwdf_cmm_clean = rwdf_cmm_clean.astype({'likes':int, 'dislikes':int})


#verificando se há valores nulos restantes. 216 valores nulos na coluna TXT
rwdf_cmm_clean.isna().sum()

#verificando registros onde a coluna txt é nula.
rwdf_cmm_clean[rwdf_cmm_clean.txt.isna()]

#coluna txt não tem informações úteis para o dataset.
#portanto será apagada.
rwdf_cmm_clean.drop('txt', axis=1, inplace=True)

In [111]:
#Criando as duas colunas com data formatada.
rwdf_cmm_clean['cDate_formated'], rwdf_cmm_clean['cDate_tmz'] = time_formatter(rwdf_cmm_clean['commentDate'])

In [112]:
#apagando a coluna de data antiga
rwdf_cmm_clean.drop('commentDate', axis=1, inplace=True)

In [113]:
#convertendo cDate_formated para datetime
rwdf_cmm_clean['cDate_formated'] = pd.to_datetime(rwdf_cmm_clean['cDate_formated'], format='%d/%b/%Y -  %H:%M:%S')
#Adicionando uma hora as datas que estejam em horario de verão.
rwdf_cmm_clean.loc[rwdf_cmm_clean['cDate_tmz'] == 'CEST', 'cDate_formated'] = rwdf_cmm_clean['cDate_formated'] + pd.Timedelta('1 hour')

In [114]:
rwdf_cmm_clean

,employee,companyAlias,commentId,likes,dislikes,cDate_formated,cDate_tmz
0,307,56aec740f1ef260003e307d6,58d018d7e010990004e38070,4,0,2017-03-20 19:00:17,CET
1,382,56aec740f1ef260003e307d6,58d0179ae010990004e3806d,1,2,2017-03-20 18:55:16,CET
2,172,56aec740f1ef260003e307d6,58cff8cde010990004e37f6a,3,0,2017-03-20 16:44:02,CET
3,135,56aec740f1ef260003e307d6,58cfefeee010990004e37f60,1,1,2017-03-20 16:06:08,CET
4,225,56aec740f1ef260003e307d6,58cfd9b4e010990004e37f52,3,2,2017-03-20 14:30:50,CET
...,...,...,...,...,...,...,...
82750,-45,58a728a0e75bda00042a3468,58ae91eb9fd38f00045eeeea,3,1,2017-02-23 08:39:59,CET
82751,118,58a728a0e75bda00042a3468,58ae85a69fd38f00045eeee4,1,0,2017-02-23 07:44:48,CET
82752,-10,58a728a0e75bda00042a3468,58ae7fa19fd38f00045eeee2,2,0,2017-02-23 07:22:12,CET
82753,-8,58a728a0e75bda00042a3468,58adbfcbb8242400048bf8c1,5,0,2017-02-22 17:43:39,CET
